Bu notebookda potansiyel duplicate olabilecek satırların bulunması hedeflenmiştir.

### Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
from pandarallel import pandarallel
from difflib import get_close_matches
pandarallel.initialize(progress_bar=True,nb_workers=24)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Read and Preprocess

In [2]:
df = pd.read_csv("../data/raw/teknofest_train_final.csv",sep="|")

In [3]:
#hard duplicateleri değil, birbirine çok yakın cümleleri tespit etmek istemiyoruz.
df = df.drop_duplicates('text').reset_index(drop=True)

### Finding Near-Duplicates

In [3]:
#en yakın 10 cümleyi bul, threshold 0.85'den büyükse dahil et
patterns = df.text.to_list()
def closeMatches(word):
    global patterns
    return get_close_matches(word,patterns,n=10,cutoff=0.85)

In [4]:
df['closest'] = df.text.parallel_apply(lambda x: closeMatches(x))

In [7]:
df_dups = df.copy()
df_dups = df_dups.explode(column=['closest']).reset_index(drop=True)

In [8]:
df_dups['self_match'] = (df_dups['text'] == df_dups['closest'])
df_dups = df_dups[df_dups['self_match']==False].drop('self_match',axis=1)
df_dups = df_dups.merge(df[['id','text','target']].rename(columns={'text':'closest'}),how='left',on='closest')

In [9]:
df_dups = df_dups[['id_x','target_x','text','id_y','target_y','closest']]

In [11]:
df_dups

,id_x,target_x,text,id_y,target_y,closest
0,d93e05f7-bfdd-4cdb-99d8-3048761b30ff,OTHER,otomasyon< sistemlerine= doğrudan bağlanabilir,d9fd2463-2ab4-443f-9e2f-ddcd8be84740,OTHER,otomasyon sistemlerine doğrudan bağlanabilir
1,c99b56c2-f683-48ac-90d4-f540f64c9328,INSULT,köpek gibi ne havlayıp duruyorsun,52093506-3a22-4614-aa40-e7ae5163fc94,INSULT,köpek gibi havlayıpcduruyorsun
2,0f512230-451f-4ccb-ba34-81570280e915,PROFANITY,ananı sikerim baban olurum,38402b8e-e23c-4871-ae97-14eeb15ea152,PROFANITY,ananı siker baban olurum
3,7d60c943-ada2-4f6f-9416-a087f8848403,RACIST,Araplar haindir.,35c47bfd-c5ab-43f8-a922-e67e0ccb446d,RACIST,araplar haindir
4,f9269767-9c1f-4b7d-80ab-84d0b8e4f4de,RACIST,Çingene ile ahbap olursan mükafatın kalbur olur,34cec706-6fed-4b67-84fe-498eda7dd6d5,RACIST,Çingeneyle ahbap olanın mükafatı kalbur olur
...,...,...,...,...,...,...
1021,0a391287-39d7-4160-8ee5-343013fab4d2,PROFANITY,orospunun çıkartıp kerhane köşelerine attığı k...,53624398-c2b4-4ee4-a55e-c5a822330f3f,PROFANITY,orospunun çıkartıp kerhane köşelerine attigi k...
1022,83d7088b-85ab-4570-8419-aee7d5e6479b,RACIST,kürt’ün kocayanına keçi güttürürler,9603c880-b2ba-46ed-935f-58379fbca9a3,RACIST,Kürdün kocayanına keçi güttürürler
1023,83d7088b-85ab-4570-8419-aee7d5e6479b,RACIST,kürt’ün kocayanına keçi güttürürler,3dead1bf-4e66-4368-b0d1-469151686bc7,RACIST,Kürdün anca kocayanına keçi güttürürler
1024,17f38489-65e0-47f3-802c-ea95e953b7f3,SEXIST,kadınsan kadınlığını bileceksin,7ed1b756-88ea-430d-91cc-8150eec51761,SEXIST,Kadın kadınlığını bilecek


In [10]:
(df_dups.target_y == df_dups.target_x).value_counts()

True     993
False     33
dtype: int64

1026 satır birbirinden farklı ancak birbine çok yakın cümleler bulduk. Bu near-duplicateslerin ya noktalama işareti ya da aynı anlamların farklı kelimelerle ifade edildiğini görmekteyiz. Bu eşleşenlerden yalnızca 33 tanesi farklı target alırken; 993 tanesi aynı targeta sahiptir.